# Jellyfish

**Inhalt:** Wie ähnlich sind zwei Strings?

**Nötige Skills:** keine

**Lernziele:** Ein paar einfache Möglichkeiten für fuzzy string matching kennenlernen

## About

Das Package: https://jellyfish.readthedocs.io/en/latest/index.html
Installation:

```bash
pip3 install jellyfish
```

## Setup

In [1]:
import pandas as pd

In [2]:
import jellyfish

## Import

Wir arbeiten mit ein paar vorbereiteten Begriffspaaren.

In [3]:
df = pd.read_csv('dataprojects/Jellyfish/Words.csv')

In [4]:
df

,Wort 1,Wort 2
0,Tisch,Tische
1,Tisch,Fisch
2,Tisch,Dorf
3,Peter Müller,Peter Mueller
4,Peter Müller,Pete Müller
5,Peter Müller,Pete Miller
6,Peter Pan,Peter Pan
7,Peter Pan,Peter V. Pan
8,Peter Pan,P. Pan
9,Peter Pan,"Pan, Peter"


## Basics

Jellyfish bietet ein paar Funktionen, um Strings zu vergleichen.

Man misst sozusagen die "Distanz" zwischen den Strings:

In [5]:
# Kleine Distanz
jellyfish.levenshtein_distance('jellyfish', 'smellyfish')

2

In [6]:
# Grosse Distanz
jellyfish.levenshtein_distance('jellyfish', 'Oxtail soup')

10

## String Comparison Methoden

Es gibt eine ganze Reihe von "Distanzmessern".

Die ganze Liste: https://jellyfish.readthedocs.io/en/latest/comparison.html

### Levenshtein-Distanz

Retourniert die Anzahl nötiger Operationen, um von einem String zum nächsten zu kommen.

In [7]:
df['Levenshtein'] = df.apply(lambda row: jellyfish.levenshtein_distance(row['Wort 1'], row['Wort 2']) ,axis=1)

In [8]:
df

,Wort 1,Wort 2,Levenshtein
0,Tisch,Tische,1
1,Tisch,Fisch,1
2,Tisch,Dorf,5
3,Peter Müller,Peter Mueller,2
4,Peter Müller,Pete Müller,1
5,Peter Müller,Pete Miller,2
6,Peter Pan,Peter Pan,0
7,Peter Pan,Peter V. Pan,3
8,Peter Pan,P. Pan,4
9,Peter Pan,"Pan, Peter",8


Der Algorithmus ist verhältnismässig doof... zB "Peter Pan" und "Pan, Peter" haben eine grosse Distanz, obwohl die Strings für für unser sehr ähnlich sind.

### Jaro-Distanz

- Erklärung: https://rosettacode.org/wiki/Jaro_distance
- Berücksichtigt die Anzahl gleicher Characters und die Anzahl der Transpositionen
- Spuckt eine Zahl zwischen 0 und 1 aus:
- 0 = komplett unterschiedlich
- 1 = komplett identisch

In [9]:
df['Jaro'] = df.apply(lambda row: jellyfish.jaro_distance(row['Wort 1'], row['Wort 2']) ,axis=1)

In [10]:
df

,Wort 1,Wort 2,Levenshtein,Jaro
0,Tisch,Tische,1,0.944444
1,Tisch,Fisch,1,0.866667
2,Tisch,Dorf,5,0.000000
3,Peter Müller,Peter Mueller,2,0.890637
4,Peter Müller,Pete Müller,1,0.972222
5,Peter Müller,Pete Miller,2,0.914141
6,Peter Pan,Peter Pan,0,1.000000
7,Peter Pan,Peter V. Pan,3,0.916667
8,Peter Pan,P. Pan,4,0.796296
9,Peter Pan,"Pan, Peter",8,0.531481


Basierend auf dieser Metrik können wir z.B. entscheiden, ob zwei Strings ein "match" sind oder nicht.

Wir sagen einfach Mal: alles über 0,8 ist für uns OK!

In [11]:
df['Jaro-Match'] = df['Jaro'] > 0.8

In [12]:
df

,Wort 1,Wort 2,Levenshtein,Jaro,Jaro-Match
0,Tisch,Tische,1,0.944444,True
1,Tisch,Fisch,1,0.866667,True
2,Tisch,Dorf,5,0.000000,False
3,Peter Müller,Peter Mueller,2,0.890637,True
4,Peter Müller,Pete Müller,1,0.972222,True
5,Peter Müller,Pete Miller,2,0.914141,True
6,Peter Pan,Peter Pan,0,1.000000,True
7,Peter Pan,Peter V. Pan,3,0.916667,True
8,Peter Pan,P. Pan,4,0.796296,False
9,Peter Pan,"Pan, Peter",8,0.531481,False


### Gängige Fehler

**False positives**
- Wörter, die als match angezeigt werden, obwohl sie kein match sind

**False negatives**
- Wörter, die nicht als match erkannt werden, obwohl sie einer sind

**Frage**: Basierend auf dem obigen Jaro-Matching, welche Matches würden Sie als false positives einstufen? Welche als false negatives?

In [13]:
# False positives:
# 

In [14]:
# False negatives:
# 


## Phonetic matching

Retourniert keine Distanzmessung, sondern eine phonetisch "vereinfachte" Version des Strings.

Die gesamte Liste: https://jellyfish.readthedocs.io/en/latest/phonetic.html

### Metaphone

" It transforms a word into a string consisting of ‘0BFHJKLMNPRSTWXY’ where ‘0’ is pronounced ‘th’ and ‘X’ is a ‘[sc]h’ sound."

Mehr: https://en.wikipedia.org/wiki/Metaphone

In [15]:
jellyfish.metaphone('John')

'JN'

In [16]:
df = pd.read_csv('dataprojects/Jellyfish/Words.csv')

In [17]:
df['Metaphone 1'] = df.apply(lambda row: jellyfish.metaphone(row['Wort 1']) ,axis=1)

In [18]:
df['Metaphone 2'] = df.apply(lambda row: jellyfish.metaphone(row['Wort 2']) ,axis=1)

In [19]:
df

,Wort 1,Wort 2,Metaphone 1,Metaphone 2
0,Tisch,Tische,TSX,TSX
1,Tisch,Fisch,TSX,FSX
2,Tisch,Dorf,TSX,TRF
3,Peter Müller,Peter Mueller,PTR MLR,PTR MLR
4,Peter Müller,Pete Müller,PTR MLR,PT MLR
5,Peter Müller,Pete Miller,PTR MLR,PT MLR
6,Peter Pan,Peter Pan,PTR PN,PTR PN
7,Peter Pan,Peter V. Pan,PTR PN,PTR F PN
8,Peter Pan,P. Pan,PTR PN,P PN
9,Peter Pan,"Pan, Peter",PTR PN,PN PTR


In [20]:
df['Metaphone-Match'] = df['Metaphone 1'] == df['Metaphone 2']

In [21]:
df

,Wort 1,Wort 2,Metaphone 1,Metaphone 2,Metaphone-Match
0,Tisch,Tische,TSX,TSX,True
1,Tisch,Fisch,TSX,FSX,False
2,Tisch,Dorf,TSX,TRF,False
3,Peter Müller,Peter Mueller,PTR MLR,PTR MLR,True
4,Peter Müller,Pete Müller,PTR MLR,PT MLR,False
5,Peter Müller,Pete Miller,PTR MLR,PT MLR,False
6,Peter Pan,Peter Pan,PTR PN,PTR PN,True
7,Peter Pan,Peter V. Pan,PTR PN,PTR F PN,False
8,Peter Pan,P. Pan,PTR PN,P PN,False
9,Peter Pan,"Pan, Peter",PTR PN,PN PTR,False


### Nysiis

"It transforms a word into a phonetic code. Like soundex and metaphone it is primarily intended for use on names (as they would be pronounced in English)."

Mehr: https://en.wikipedia.org/wiki/New_York_State_Identification_and_Intelligence_System

Uses only the 1st substring!

In [22]:
df['Nysiis 1'] = df.apply(lambda row: jellyfish.nysiis(row['Wort 1']) ,axis=1)

In [23]:
df['Nysiis 2'] = df.apply(lambda row: jellyfish.nysiis(row['Wort 2']) ,axis=1)

In [24]:
df['Nysiis-Match'] = df['Nysiis 1'] == df['Nysiis 2']

In [25]:
df

,Wort 1,Wort 2,Metaphone 1,Metaphone 2,Metaphone-Match,Nysiis 1,Nysiis 2,Nysiis-Match
0,Tisch,Tische,TSX,TSX,True,TAS,TASS,False
1,Tisch,Fisch,TSX,FSX,False,TAS,FAS,False
2,Tisch,Dorf,TSX,TRF,False,TAS,DARF,False
3,Peter Müller,Peter Mueller,PTR MLR,PTR MLR,True,PATAR,PATAR,True
4,Peter Müller,Pete Müller,PTR MLR,PT MLR,False,PATAR,PAT,False
5,Peter Müller,Pete Miller,PTR MLR,PT MLR,False,PATAR,PAT,False
6,Peter Pan,Peter Pan,PTR PN,PTR PN,True,PATAR,PATAR,True
7,Peter Pan,Peter V. Pan,PTR PN,PTR F PN,False,PATAR,PATAR,True
8,Peter Pan,P. Pan,PTR PN,P PN,False,PATAR,P.,False
9,Peter Pan,"Pan, Peter",PTR PN,PN PTR,False,PATAR,"PAN,",False


## Fazit

- Kann hilfreich sein (zB um einzelne Tippfehler zu spotten)
- Aber es reicht nicht extrem weit
- Schwierig, im jeweilgen Fall den richtigen Algorithmus auszuwählen
- Gefahr false positives / false negatives immer vorhanden.